In [2]:
from op_analytics.cli.subcommands.pulls.defillama.dataaccess import DefiLlama

In [2]:
## EXAMPLE 1: Register remote parquet files as a view.

duckdb_client = DefiLlama.PROTOCOLS_TOKEN_TVL.read(min_date="2024-10-01")

print(duckdb_client.sql("DESCRIBE protocols_token_tvl_v1"))

# An aggregation that runs on top of the remote data.
# Takes ~5s to run, not to bad since we are only looking at dates starting 2024-10-01.
print(duckdb_client.sql(
"""
SELECT 
    protocol_slug, 
    chain, 
    token, 
    sum(app_token_tvl_usd) AS tvl_usd, 
    count(distinct dt) AS num_dates
FROM protocols_token_tvl_v1
GROUP BY 1, 2, 3
ORDER BY 4 DESC
LIMIT 10
"""))


2024-11-26 17:34:36 [info     ] loaded vault from .env file    filename=vault.py lineno=32
2024-11-26 17:34:36 [debug    ] loaded vault: 17 items         filename=vault.py lineno=76
2024-11-26 17:34:36 [info     ] querying markers for 'protocols_token_tvl_v1' DateFilter(min_date=datetime.date(2024, 10, 1), max_date=None, datevals=None) filename=dataaccess.py lineno=194
2024-11-26 17:34:36 [debug    ] connecting to OPLABS Clickhouse client... filename=client.py lineno=26
2024-11-26 17:34:37 [debug    ] initialized OPLABS Clickhouse client. filename=client.py lineno=38
2024-11-26 17:34:37 [info     ] 57 markers found               filename=dataaccess.py lineno=208
2024-11-26 17:34:38 [info     ] registered view 'protocols_token_tvl_v1' using 57 parquet paths filename=dataaccess.py lineno=220
┌────────────────────────┐
│          name          │
│        varchar         │
├────────────────────────┤
│ protocols_token_tvl_v1 │
└────────────────────────┘

┌───────────────────┬─────────────┬─

In [3]:
## EXAMPLE 2: Dump the entire dataset locally

DefiLlama.PROTOCOLS_TOKEN_TVL.dump_local_copy()

2024-11-26 17:34:45 [info     ] registered view 'protocols_token_tvl_v1' using uri wildcard 'gs://oplabs-tools-data-sink/defillama/protocols_token_tvl_v1/dt=*/out.parquet' filename=dataaccess.py lineno=220
┌────────────────────────┐
│          name          │
│        varchar         │
├────────────────────────┤
│ protocols_token_tvl_v1 │
└────────────────────────┘

2024-11-26 17:34:45 [info     ] begin dump copy to '/Users/pedro/github/ethereum-optimism/op-analytics-alt/parquet/defillama/protocols_token_tvl_v1__20241126T223445.parquet' filename=dataaccess.py lineno=66
2024-11-26 17:37:06 [info     ] done dump copy to '/Users/pedro/github/ethereum-optimism/op-analytics-alt/parquet/defillama/protocols_token_tvl_v1__20241126T223445.parquet' filename=dataaccess.py lineno=68
┌─────────────────────────────────────────┐
│                  name                   │
│                 varchar                 │
├─────────────────────────────────────────┤
│ protocols_token_tvl_v1                  

In [ ]:
## EXAMPLE 2 (continuud)

## Use the local copy directly by providing the exact fetch timestamp
duckdb_client = DefiLlama.PROTOCOLS_TOKEN_TVL.load_local_copy(timestamp="20241126T220908")


2024-11-26 17:37:06 [info     ] load copy from '/Users/pedro/github/ethereum-optimism/op-analytics-alt/parquet/defillama/protocols_token_tvl_v1__20241126T220908.parquet' filename=dataaccess.py lineno=88
┌─────────────────────────────────────────┐
│                  name                   │
│                 varchar                 │
├─────────────────────────────────────────┤
│ protocols_token_tvl_v1                  │
│ protocols_token_tvl_v1__20241126T220908 │
│ protocols_token_tvl_v1__20241126T223445 │
└─────────────────────────────────────────┘



In [5]:
print(duckdb_client.sql(
"""
SELECT 
    min(dt)
FROM protocols_token_tvl_v1__20241126T220908
"""))

┌────────────┐
│  min(dt)   │
│    date    │
├────────────┤
│ 2018-04-27 │
└────────────┘



In [6]:
# An aggregation that runs on top of the local copy.
# Takes ~3s to run (this includes ALL dates since 2018-04-27)
# Running this on the remote dataset would take longer.
print(duckdb_client.sql(
"""
SELECT 
    protocol_slug, 
    chain, 
    token, 
    max(app_token_tvl_usd) AS max_tvl_usd, 
    arg_max(dt, app_token_tvl_usd) as max_tvl_usd_dt,
    count(distinct dt) AS num_dates
FROM protocols_token_tvl_v1__20241126T220908
GROUP BY 1, 2, 3
ORDER BY 4 DESC
LIMIT 10
"""))

┌───────────────┬──────────────────┬─────────┬────────────────────┬────────────────┬───────────┐
│ protocol_slug │      chain       │  token  │    max_tvl_usd     │ max_tvl_usd_dt │ num_dates │
│    varchar    │     varchar      │ varchar │       double       │      date      │   int64   │
├───────────────┼──────────────────┼─────────┼────────────────────┼────────────────┼───────────┤
│ hotbit        │ Tron             │ TRX     │     728037762725.0 │ 2024-01-31     │       614 │
│ iron-finance  │ Polygon          │ TITAN   │ 244941123692.78592 │ 2021-06-16     │         4 │
│ binance-cex   │ Bitcoin          │ BTC     │  58700225047.75569 │ 2024-11-22     │       744 │
│ lido          │ Ethereum         │ WETH    │   40177918659.3112 │ 2024-03-12     │      1274 │
│ sushiswap     │ Polygon          │ TITAN   │  35896427732.67128 │ 2021-06-17     │       301 │
│ sushiswap     │ Harmony          │ RVRS    │  35486417432.64003 │ 2022-06-13     │       157 │
│ hex           │ staking     

In [7]:
## EXAMPLE 2 (continuud)

## Don't provide a timestamp, gives you the latest fetched copy.
duckdb_client = DefiLlama.PROTOCOLS_TOKEN_TVL.load_local_copy()

print(duckdb_client.sql(
"""
SELECT 
    min(dt)
FROM protocols_token_tvl_v1__20241126T220908
"""))

2024-11-26 17:37:09 [info     ] load copy from '/Users/pedro/github/ethereum-optimism/op-analytics-alt/parquet/defillama/protocols_token_tvl_v1__20241126T223445.parquet' filename=dataaccess.py lineno=88
┌─────────────────────────────────────────┐
│                  name                   │
│                 varchar                 │
├─────────────────────────────────────────┤
│ protocols_token_tvl_v1                  │
│ protocols_token_tvl_v1__20241126T220908 │
│ protocols_token_tvl_v1__20241126T223445 │
└─────────────────────────────────────────┘

┌────────────┐
│  min(dt)   │
│    date    │
├────────────┤
│ 2018-04-27 │
└────────────┘



In [8]:
## EXAMPLE 3: Dump only a date range to a local parquet file

DefiLlama.PROTOCOLS_TOKEN_TVL.dump_local_copy(min_date="2024-06-01")

2024-11-26 17:37:09 [info     ] querying markers for 'protocols_token_tvl_v1' DateFilter(min_date=datetime.date(2024, 6, 1), max_date=None, datevals=None) filename=dataaccess.py lineno=194
2024-11-26 17:37:09 [info     ] 179 markers found              filename=dataaccess.py lineno=208
2024-11-26 17:37:10 [info     ] registered view 'protocols_token_tvl_v1' using 179 parquet paths filename=dataaccess.py lineno=220
┌─────────────────────────────────────────┐
│                  name                   │
│                 varchar                 │
├─────────────────────────────────────────┤
│ protocols_token_tvl_v1                  │
│ protocols_token_tvl_v1__20241126T220908 │
│ protocols_token_tvl_v1__20241126T223445 │
└─────────────────────────────────────────┘

2024-11-26 17:37:10 [info     ] begin dump copy to '/Users/pedro/github/ethereum-optimism/op-analytics-alt/parquet/defillama/protocols_token_tvl_v1__20241126T223710.parquet' filename=dataaccess.py lineno=66
2024-11-26 17:37:17 [in

In [9]:


print(duckdb_client.sql(
"""
SELECT 
    protocol_slug, 
    chain, 
    token, 
    max(app_token_tvl_usd) AS max_tvl_usd, 
    arg_max(dt, app_token_tvl_usd) as max_tvl_usd_dt,
    count(distinct dt) AS num_dates
FROM protocols_token_tvl_v1__20241126T220908
GROUP BY 1, 2, 3
ORDER BY 4 DESC
LIMIT 10
"""))

┌───────────────┬──────────────────┬─────────┬────────────────────┬────────────────┬───────────┐
│ protocol_slug │      chain       │  token  │    max_tvl_usd     │ max_tvl_usd_dt │ num_dates │
│    varchar    │     varchar      │ varchar │       double       │      date      │   int64   │
├───────────────┼──────────────────┼─────────┼────────────────────┼────────────────┼───────────┤
│ hotbit        │ Tron             │ TRX     │     728037762725.0 │ 2024-01-31     │       614 │
│ iron-finance  │ Polygon          │ TITAN   │ 244941123692.78592 │ 2021-06-16     │         4 │
│ binance-cex   │ Bitcoin          │ BTC     │  58700225047.75569 │ 2024-11-22     │       744 │
│ lido          │ Ethereum         │ WETH    │   40177918659.3112 │ 2024-03-12     │      1274 │
│ sushiswap     │ Polygon          │ TITAN   │  35896427732.67128 │ 2021-06-17     │       301 │
│ sushiswap     │ Harmony          │ RVRS    │  35486417432.64003 │ 2022-06-13     │       157 │
│ hex           │ staking     

In [3]:
## EXAMPLE 4: Read metadata. 
# 
# The metadata df is written out entirely for each date
# So th read it on must pick a single date.

duckdb_client = DefiLlama.PROTOCOLS_METADATA.read(date_range_spec="@20241126:+1")

2024-11-26 18:06:32 [info     ] loaded vault from .env file    filename=vault.py lineno=32
2024-11-26 18:06:32 [debug    ] loaded vault: 17 items         filename=vault.py lineno=76
2024-11-26 18:06:32 [info     ] querying markers for 'protocols_metadata_v1' DateFilter(min_date=None, max_date=None, datevals=[datetime.date(2024, 11, 26)]) filename=dataaccess.py lineno=202
2024-11-26 18:06:32 [debug    ] connecting to OPLABS Clickhouse client... filename=client.py lineno=26
2024-11-26 18:06:32 [debug    ] initialized OPLABS Clickhouse client. filename=client.py lineno=38
2024-11-26 18:06:32 [info     ] 1 markers found                filename=dataaccess.py lineno=216
2024-11-26 18:06:33 [info     ] registered view 'protocols_metadata_v1' using 1 parquet paths filename=dataaccess.py lineno=228
┌───────────────────────┐
│         name          │
│        varchar        │
├───────────────────────┤
│ protocols_metadata_v1 │
└───────────────────────┘



In [4]:
duckdb_client.sql("SELECT * FROM protocols_metadata_v1 LIMIT 10")

┌─────────────────┬─────────────────┬───────────────────┬─────────────────┬─────────────────┬───────────────────────┬────────────┐
│  protocol_name  │  protocol_slug  │ protocol_category │ parent_protocol │ wrong_liquidity │ misrepresented_tokens │     dt     │
│     varchar     │     varchar     │      varchar      │     varchar     │     boolean     │        boolean        │    date    │
├─────────────────┼─────────────────┼───────────────────┼─────────────────┼─────────────────┼───────────────────────┼────────────┤
│ Binance CEX     │ binance-cex     │ CEX               │ binance-cex     │ NULL            │ NULL                  │ 2024-11-26 │
│ Lido            │ lido            │ Liquid Staking    │ lido            │ NULL            │ NULL                  │ 2024-11-26 │
│ Bitfinex        │ bitfinex        │ CEX               │ bitfinex        │ NULL            │ NULL                  │ 2024-11-26 │
│ OKX             │ okx             │ CEX               │ okx             │ NULL   